In [1]:
import numpy as np
import pandas as pd
import sklearn
import numpy.ma as ma
import random
import time
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.linear_model import LinearRegression

C:\Users\Gabriele\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [2]:
ratings = pd.read_csv(
   'ratings.dat',
    sep = "::",
    names = ['UserID', 'MovieID', 'Rating', 'Timestamp'],
    encoding='latin-1',
    engine='python',
)

ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## 1.1 Naive Methods


### 1.1.1 Global average rating

In [3]:
## Global rating method
start = time.time()
random.seed(3)

x = ratings.sample(frac = 1)
X = np.array_split(x, 5) #Split dataset into 5 random subsets for Cross Validation

rmses = []
maes = []

for i in range(5):
    X_test = X[i] #Test set
    X_train = pd.concat([X[j] for j in range(5) if j!=i]) #Combine 4 subsets for training set
    average_score = np.mean(X_train['Rating']) #Compute average score of training set
    reccomendations = np.array([average_score]*len(X_test)) #Predicted rating
    rmse = np.sqrt(np.mean((reccomendations - X_test['Rating'])**2))
    rmses += [rmse]
    mae = np.mean(abs(reccomendations - X_test['Rating']))
    maes += [mae]

end = time.time()
print('The runtime of the algorithm is: {}'.format(round(end - start,3)))
print('The RMSE is: {}'.format(round(np.mean(rmses),5)))
print('The MAE is: {}'.format(round(np.mean(maes),5)))

The runtime of the algorithm is: 0.72
The RMSE is: 1.1171
The MAE is: 0.93386


### 1.1.2 User average rating

In [4]:
## Per user rating
start = time.time()
random.seed(3)

x = ratings.sample(frac = 1)
X = np.array_split(x, 5) #Split dataset into 5 random subsets for Cross Validation

rmses = []
maes = []

for i in range(5):
    X_test = X[i] #Test set
    X_train = pd.concat([X[j] for j in range(5) if j!=i]) #Combine 4 subsets for training set
    reccomendations=[]
    rat=dict.fromkeys(np.unique(X_train['UserID']))
    
    for user in np.unique(X_train['UserID']):
        rat[user]=X_train.loc[X_train['UserID']==user]['Rating'].mean() #for each user in the training set compute the mean rating
    for user in X_test['UserID']:
        if user in rat:
            reccomendations.append(rat[user]) #For each user in the test set, take its avreage rating from the training set
        else:
            reccomendations.append(np.mean(X_train['Rating'])) #If there is no occurence of the user in the training set, use global average
    
    rmse = np.sqrt(np.mean((reccomendations - X_test['Rating'])**2))
    rmses += [rmse]
    mae = np.mean(abs(reccomendations - X_test['Rating']))
    maes += [mae]
    
end = time.time()
print('The runtime of the algorithm is: {}'.format(round(end - start,3)))
print('The RMSE is: {}'.format(round(np.mean(rmses),5)))
print('The MAE is: {}'.format(round(np.mean(maes),5)))

The runtime of the algorithm is: 82.987
The RMSE is: 1.03556
The MAE is: 0.82909


### 1.1.3 Movie average rating

In [ ]:
## Per item rating
start = time.time()
random.seed(3)

x = ratings.sample(frac = 1)
X = np.array_split(x, 5) #Split dataset into 5 random subsets for Cross Validation

rmses = []
maes = []

for i in range(5):
    X_test = X[i] #Test set
    X_train = pd.concat([X[j] for j in range(5) if j!=i]) #Combine 4 subsets for training set
    reccomendations=[]
    rat=dict.fromkeys(np.unique(X_train['MovieID']))
    
    for movie in np.unique(X_train['MovieID']):
        rat[movie]=X_train.loc[X_train['MovieID']==movie]['Rating'].mean() #for each movie in the training set compute the mean rating
    for movie in X_test['MovieID']:
        if movie in rat:
            reccomendations.append(rat[movie]) #For each movie in the test set, take its avreage rating from the training set
        else:
            reccomendations.append(np.mean(X_train['Rating'])) #If there is no occurence of the movie in the training set, use global average
    
    rmse = np.sqrt(np.mean((reccomendations - X_test['Rating'])**2))
    rmses += [rmse]
    mae = np.mean(abs(reccomendations - X_test['Rating']))
    maes += [mae]
    
end = time.time()
print('The runtime of the algorithm is: {}'.format(round(end - start,3)))
print('The RMSE is: {}'.format(round(np.mean(rmses),5)))
print('The MAE is: {}'.format(round(np.mean(maes),5)))

### 1.1.4/5 Linear regression with and without coefficient

In [ ]:
##Linear regression

start = time.time()
random.seed(3)

x = ratings.sample(frac = 1)
X = np.array_split(x, 5) #Split dataset into 5 random subsets for Cross Validation
simrmses = [] #simple linear rigression = no intercept
simmaes = []
rmses = [] #linear regression with intercept
maes = []

for i in range(5):
    X_test = X[i] #Test set
    X_train = pd.concat([X[j] for j in range(5) if j!=i]) #Combine 4 subsets for training set
    
    train=X_train.pivot(index='MovieID', columns='UserID', values='Rating') #Training set into matrix form
    colavg=train.mean() #User average
    rowavg=train.mean(axis=1) #Movie average
    average_score = np.mean(X_train['Rating']) #Average score (in case missing values for test set)
    
    indexes=train[train.notnull()].stack().index #Indexes of all the non-NA values
    lr=[]
    for k in indexes:
        lr.append((rowavg[k[0]],colavg[k[1]],train.loc[k[0],k[1]])) #Create a list of (Movie_avg, User_avg, Rating)
        
    df = pd.DataFrame(lr, columns = ['x1', 'x2','y'])
    simplereg=LinearRegression(fit_intercept=False).fit(df.iloc[:,0:2],df.iloc[:,2]) #Linear regression without intercept
    reg=LinearRegression().fit(df.iloc[:,0:2],df.iloc[:,2]) #Linear regression with intercept
    
    for i in range(ratings['MovieID'].max()):
        if (i not in rowavg):
            rowavg[i]=average_score #If in the test set there is no movie average, use global average
    for i in range(ratings['UserID'].max()):
        if (i not in colavg):
            colavg[i]=average_score #If in the test set there is no user average, use global average
            
    #Recommendation computed with linear regression without intercept
    simplereccom = simplereg.coef_[0]*rowavg[X_test['MovieID']].reset_index()[0]+simplereg.coef_[1]*colavg[X_test['UserID']].reset_index()[0]
    #Recommendation computed with linear regression with intercept
    reccom = reg.intercept_+reg.coef_[0]*rowavg[X_test['MovieID']].reset_index()[0]+reg.coef_[1]*colavg[X_test['UserID']].reset_index()[0]
    
    #Values higher than 5 are rounded to 5 and values smaller than 1 are rounded to 1
    for i in range(len(simplereccom)-1):
        if (simplereccom[i]>5):
            simplereccom[i]=5
        elif (simplereccom[i]<1):
            simplereccom[i]=1
    
    for i in range(len(reccom)-1):
        if (reccom[i]>5):
            reccom[i]=5
        elif (reccom[i]<1):
            reccom[i]=1
        
    #RMSE and MAE for linear regression without intercept
    simrmse = np.sqrt(np.mean((simplereccom - X_test['Rating'].reset_index()['Rating'])**2))
    simrmses += [simrmse]
    simmae = np.mean(abs(simplereccom - X_test['Rating'].reset_index()['Rating']))
    simmaes += [simmae]
    
    #RMSE and MAE for linear regression with intercept
    rmse = np.sqrt(np.mean((reccom - X_test['Rating'].reset_index()['Rating'])**2))
    rmses += [rmse]
    mae = np.mean(abs(reccom - X_test['Rating'].reset_index()['Rating']))
    maes += [mae]
    
end = time.time()
print('The runtime of the algorithm is: {}'.format(round(end - start,3)))
print('The RMSE of the linear regression without intercept is: {}'.format(round(np.mean(simrmses),5)))
print('The MAE of the linear regression without intercept is: {}'.format(round(np.mean(simmaes),5)))
print('The RMSE of the linear regression with intercept is: {}'.format(round(np.mean(rmses),5)))
print('The MAE of the linear regression with intercept is: {}'.format(round(np.mean(maes),5)))

## 1.2 UV Matrix Decomposition

Notes: To increase our chances of finding the global minimum, we need to pick many dif- ferent starting points, that is, different choices of the initial matrices U and V . However, there is never a guarantee that our best local minimum will be the global minimum.

In [1]:
class Matrix_Decomposition:
    def __init__(self, matrix, dimensions = 5, iterations = 1):
        self.matrix = matrix
        self.n = self.matrix.shape[0]
        self.m = self.matrix.shape[1]
        self.d = dimensions
        self.normalized = self.matrix - np.nanmean(self.matrix,axis=1, keepdims=True)
        self.normalized = self.matrix - np.nanmean(self.matrix,axis=0, keepdims=True)
        self.U = np.random.rand(self.n,self.d) + np.sqrt(np.nanmean(self.matrix)/self.d)
        self.V = np.random.rand(self.d,self.m) + np.sqrt(np.nanmean(self.matrix)/self.d)
        self.iter = iterations
        self.transformed_UV = np.dot(self.U, self.V)+np.nanmean(self.matrix,axis=0, keepdims=True)+np.nanmean(self.matrix,axis=1, keepdims=True)
            
    def rmse(self):
        UV = np.dot(self.U,self.V)
        error = np.nanmean((self.normalized - UV)**2)
        return error


    def UV(self):
        start = time.time()
        err = [self.rmse()]
        for i in range(self.iter):
            # decompose user matrix
            for r in range(0,self.n):
                for s in range(0,self.d):
                    m = np.array(self.normalized[r,:])
                    v = np.array(self.V[s,:])
                    u = np.array(self.U[r,:])
                    index_no_s = np.array(range(self.d)) != s
                    self.U[r,s]=np.nansum(v*(m-np.dot(u[index_no_s],self.V[index_no_s,:])))/np.sum(v**2)
            
            # decompose item matrix
            for s in range(0,self.m):
                for r in range(0,self.d):
                    m = np.array(self.normalized[:,s])
                    u = np.array(self.U[:,r])
                    v = np.array(self.V[:,s])
                    index_no_r = np.array(range(self.d)) != r
                    self.V[r,s]=np.nansum(u * (m-np.dot(self.U[:,index_no_r],v[index_no_r])))/np.sum(u**2)
                    
            if err[i] - self.rmse() < 0.000001 and i > self.iter/2:
                break
            
            else:
                print('Iteration: ' + str(i+1) + ', Error: ' + str(self.rmse_new()))
                err.append(self.rmse_new())

        end = time.time()
        print('The runtime of the algorithm is: {}'.format(round(end - start,3)))

        return self.U, self.V, err

In [ ]:
# testing runtime for different number of iterations
random.seed(3)
models = dict()
for i in range (5):
    models[i] = Matrix_Decomposition(M, iterations = i, dimensions = 5)

for i in range (5):
    print("Iteration: " + str(i))
    models[i].UV()

In [ ]:
# testing error behaviour for different dimensions
random.seed(3)
models = dict()
for i in range (10):
    models[i] = Matrix_Decomposition(M, iterations = 5, dimensions = i)

for i in range (10):
    print("Dimension: " + str(i+1))
    models[i].UV()

In [ ]:
## Prediction of ratings
random.seed(3)
start = time.time()

x = ratings.sample(frac = 1)
X = np.array_split(x, 5)

rmses = []
maes = []

for i in range(5):
    # Find train and test utility matrices
    X_test = X[i]
    X_train = pd.concat([X[j] for j in range(5) if j!=i])
    Y_test = X_test.copy()
    Y_train = X_train.copy()
    Y_train[['Rating']] = np.nan
    X_test = pd.concat([X_test,Y_train])
    Y_test[['Rating']] = np.nan
    X_train = pd.concat([X_train, Y_test])
    
    # Perform prediction
    Utility_DF = X_train.pivot(index = 'UserID', columns ='MovieID', values = 'Rating')
    Utility_test = X_test.pivot(index = 'UserID', columns ='MovieID', values = 'Rating')
    
    M = Utility_DF.to_numpy()
    M_test = Utility_test.to_numpy()
    model_matrix = Matrix_Decomposition(M, iterations = 90)
    uv = model_matrix.UV()
    r=np.dot(model_matrix.U, model_matrix.V)+np.nanmean(M,axis=0, keepdims=True)+np.nanmean(M,axis=1, keepdims=True)
    rmse = np.sqrt(np.nanmean((r - M_test)**2))
    rmses += [rmse]
    mae = np.nanmean(abs(r - M_test))
    maes += [mae]
    
end = time.time()
print('The runtime of the algorithm is: {}'.format(round(end - start,3)))

In [ ]:
r=np.dot(model_matrix.U, model_matrix.V)+np.nanmean(M,axis=0, keepdims=True)+np.nanmean(M,axis=1, keepdims=True)

In [ ]:
np.sqrt(np.nanmean((M-r)**2))

## 1.3 Matrix Factorization

We firstly show the code without Cross Validation, this is the one we have then used for the second part of the code. 
In the second block, Cross validation is applied to Matrix factorization too, unfortunately this code too long to run so we have preferred to use the former for the second part. 

In [ ]:
#MATRIX FACTORIZATION WITHOUT CROSS VALIDATION

start = time.time()
random.seed(3)

# Parameters reported on Mymedialite website
learn_rate=0.005
reg=0.05
num_factors=10
num_iter=75

x = ratings.pivot(index='MovieID', columns='UserID', values='Rating') 
M=np.random.rand(x.shape[0],num_factors) #Initialize the weights in M randomly (the number of rows is the number of movies, the number of columns is 10: num_factors)
M=pd.DataFrame(M, index=x.index) 
U=np.random.rand(num_factors,x.shape[1]) #Initialize the weights in U randomly (the number of columns is the number of users, the number of rows is 10: num_factors)
U=pd.DataFrame(U, columns=x.columns)

indexes=x[x.notnull()].stack().index
iterat=1
rmse=100
newrmse=99

while (iterat < num_iter and newrmse-rmse < 0.01 ): #Matrix Factorization algorithm
    print('iteration: ' + str(iterat))
    iterat += 1
    for k in indexes: #iterate over each known element of X
        movie = M.loc[k[0],]
        user = U.loc[:,k[1]]
        x_hat = np.dot(movie,user) #Predicted rating
        error = x.loc[k[0],k[1]] - x_hat #Compute the training error
        M.loc[k[0],] = movie + learn_rate * (2*error*user - reg*movie) #Update the corresponding row of M based on the equation from the paper
        U.loc[:,k[1]] = user + learn_rate * (2*error*movie - reg*user) #Update the corresponding columns of U based on the equation from the paper
            
    factoriz=pd.DataFrame(np.matmul(M, U), index=x.index, columns= x.columns)
    rmse=newrmse
    newrmse=np.sqrt(((factoriz-x)**2).mean().mean()) #Compute the new RMSE, and check whether it has decreased before next loop
            
mae = abs(factoriz-x).mean().mean()

end = time.time()
print('The runtime of the algorithm is: {}'.format(round(end - start,3)))
print('The RMSE of the linear regression without intercept is: {}'.format(round(newrmse,5)))
print('The MAE of the linear regression without intercept is: {}'.format(round(mae,5)))

In [ ]:
#MATRIX FACTORIZATION WITH CROSS VALIDATION

start = time.time()
random.seed(3)

x = ratings.sample(frac = 1) #Split dataset into 5 random subsets for Cross Validation
X = np.array_split(x, 5)

# Parameters reported on Mymedialite website
learn_rate=0.005
reg=0.05
num_factors=10
num_iter=75

rmses = []
maes = []

for i in range(5):
    print(i)
    X_test = X[i]
    X_train = pd.concat([X[j] for j in range(5) if j!=i])
    train=X_train.pivot(index='MovieID', columns='UserID', values='Rating')
    
    M=np.random.rand(train.shape[0],num_factors) #Initialize the weights in M randomly (the number of rows is the number of movies, the number of columns is 10: num_factors)
    M=pd.DataFrame(M, index=train.index)
    U=np.random.rand(num_factors,train.shape[1]) #Initialize the weights in U randomly (the number of columns is the number of users, the number of rows is 10: num_factors)
    U=pd.DataFrame(U, columns=train.columns)
    
    indexes=train[train.notnull()].stack().index
    iterat=1
    rmse=100
    newrmse=99
    
    while (iterat < num_iter and newrmse-rmse < 0.01 ): #Matrix Factorization algorithm
        iterat+=1
        for k in indexes: #iterate over each known element of X
            movie = M.loc[k[0],]
            user = U.loc[:,k[1]]
            x_hat = np.dot(movie,user) #Predicted rating
            error = train.loc[k[0],k[1]] - x_hat #Compute the training error
            M.loc[k[0],] = movie + learn_rate * (2*error*user - reg*movie) #Update the corresponding row of M based on the equation from the paper
            U.loc[:,k[1]] = user + learn_rate * (2*error*movie - reg*user) #Update the corresponding columns of U based on the equation from the paper
            
        factoriz=pd.DataFrame(np.matmul(M, U), index=train.index, columns= train.columns)
        rmse=newrmse
        newrmse=np.sqrt(np.mean((factoriz[factoriz.columns & test.columns]-test)**2))
            
    #rmse = np.sqrt(np.mean((np.matmul(M, U)-train)**2))
    rmses += [newrmse]
    mae = np.mean(abs(factoriz[factoriz.columns & test.columns]-test))
    maes += [mae]
    
end = time.time()
print('The runtime of the algorithm is: {}'.format(round(end - start,3)))
print('The RMSE is: {}'.format(round(np.mean(rmses),5)))
print('The MAE is: {}'.format(round(np.mean(maes),5)))